In [1]:
## build researcher events, student events, and self-study events
## researcher are simply those with NH citations

In [2]:
import sys

import pandas as pd
import time
import datetime

In [3]:
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nh_metrics_db = application.new_db_engine('nanohub_metrics')
wang159_myrmekes_db = application.new_db_engine('wang159_myrmekes')

salesforce = application.new_salesforce_engine()
db_s = salesforce

2021-07-22 17:30:04,928| WARNING | Could not read SSH configuration file: ~/.ssh/config


2021-07-22 17:30:04,928 - [WARNING] sshtunnel.SSHTunnelForwarder [sshtunnel._read_ssh_config:1032]: Could not read SSH configuration file: ~/.ssh/config


2021-07-22 17:30:04,934| INFO    | 0 keys loaded from agent


2021-07-22 17:30:04,934 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_agent_keys:1060]: 0 keys loaded from agent


2021-07-22 17:30:04,941| INFO    | 0 key(s) loaded


...

2021-07-22 17:30:05,630 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_keys:1117]: 0 key(s) loaded


2021-07-22 17:30:05,637| INFO    | Connecting to gateway: db2.nanohub.org:22 as user 'saxenap'


2021-07-22 17:30:05,637 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.__init__:978]: Connecting to gateway: db2.nanohub.org:22 as user 'saxenap'
2021-07-22 17:30:05,646 - [INFO] nanoHUB.containers.dataaccess [connection.get_connection_for:78]: Started SSH Tunnel with db2.nanohub.org


2021-07-22 17:30:05,861| INFO    | Opening tunnel: 0.0.0.0:46683 <> 127.0.0.1:3306


2021-07-22 17:30:05,861 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel._serve_forever_wrapper:1433]: Opening tunnel: 0.0.0.0:46683 <> 127.0.0.1:3306
Obtained Salesforce access token ...... True


## Nanohub user growth

In [4]:
## need 
# 1) jos_users from nanohub with id,name,username,email,registerDate,lastvisitDate -> registered users
# 2) toolstart df filtered by username -> simulation users

# x-axis: each month from 2000 to 2019

# y-axis: 12 month trailing sum

# simulation users: unique users who have run at least 1 simulation

# registered users: new user registeration

# visitor count: unique IP address

In [ ]:
## build jos_users_full
sql_query = "select id,name,username,email,registerDate,lastvisitDate from jos_users order by registerDate desc"
jos_users_full = pd.read_sql_query(sql_query, nanohub_db)
display(jos_users_full.head(2))
print(jos_users_full.shape)

In [ ]:
## build all users list
sql_query = "select datetime,ip,countryip,user,tool from toolstart where user != 'instanton' "\
+"and user != 'gridstat' and datetime <= '2002-01-01' and datetime >= '0000-01-01' order by datetime desc"
toolstart_df = pd.read_sql_query(sql_query, nh_metrics_db)
display(toolstart_df.head(2))
print(toolstart_df.shape)

In [ ]:
## convert the dataframes to datetime
jos_users_full['registerDate'] = pd.to_datetime(jos_users_full.registerDate)
jos_users_full['lastvisitDate'] = pd.to_datetime(jos_users_full.lastvisitDate)

toolstart_df['datetime'] = pd.to_datetime(toolstart_df.datetime)

In [ ]:
all_months_list = list()
for this_year in range(2000,2021+1):
    for this_month in range(1,12+1):
        all_months_list.append(datetime.datetime(this_year, this_month, 1))
        
for this_year in range(2021,2022):
    for this_month in range(1,2):
        all_months_list.append(datetime.datetime(this_year, this_month, 1))
        
months_bin = pd.DataFrame(all_months_list, columns=['year_month'])

# display
months_bin.head(2)

In [ ]:
## jos_users_full registration cumulative monthly
months_bin['registered_12_trail'] = months_bin.year_month.apply(lambda x:  \
                                jos_users_full.username[(jos_users_full.registerDate < x)\
                            &(jos_users_full.registerDate > (x-datetime.timedelta(days=365)))].nunique() )

In [ ]:
##the all users must be done in a bi-yearly basis
months_bin['sim_12_trail'] = months_bin.year_month.apply(lambda x: toolstart_df.user[(toolstart_df.datetime < x) \
                                        &(toolstart_df.datetime > (x-datetime.timedelta(days=365)))].nunique() )

In [ ]:
import datetime

In [ ]:
print(months_bin['year_month'].to_list().index(pd.to_datetime('2002-01-01')) )
temp = months_bin['sim_12_trail']#[:25]

In [ ]:
print(temp[-5:])
print(temp[20:25])

In [ ]:
## computer has memory limits, so split the sim_12_trail into batches
start_year = 2002 #2002 #allows for flexibility as well for future updates

while start_year < 2021:#datetime.datetime.now().year:
    prev_temp_ind = months_bin['year_month'].to_list().index(pd.to_datetime(str(start_year)+'-01-01')) 
    
    start_year += 1
    end_date = r"'"+str(start_year)+r"-01-01'"
#     print(end_date)
    start_date = r"'"+str(start_year-2)+r"-01-01'"
    
    sql_query = "select datetime,ip,countryip,user,tool from toolstart where user != 'instanton' "\
        +"and user != 'gridstat' and datetime <= "+end_date+" and datetime >= "+start_date+" order by datetime desc"
    toolstart_df = pd.read_sql_query(sql_query, engine_metrics)
    toolstart_df['datetime'] = pd.to_datetime(toolstart_df.datetime)
    print(toolstart_df.shape)
    
    temp2 = months_bin.year_month.apply(lambda x: toolstart_df.user[(toolstart_df.datetime < x) \
                                        &(toolstart_df.datetime > (x-datetime.timedelta(days=365)))].nunique() )
    
    max_temp_ind = months_bin['year_month'].to_list().index(pd.to_datetime(str(start_year)+'-01-01')) 
    
    temp[prev_temp_ind:max_temp_ind] = temp2[prev_temp_ind:max_temp_ind]

In [ ]:
# calculating total number of visitors
import json
with open(cwd+'/visitors.json') as f:
    visitor_dict = f.read()
    visitor_dict = visitor_dict.replace("\'",'\"')
    visitor_dict = json.loads(visitor_dict)
#     visitor_dict.replace(r"'",r'"')
#     vicitor_dict = json.load(visitor_dict)

In [ ]:
import re
pattern = re.compile('\d+]')
step1 = re.findall(pattern,visitor_dict['data'])
res_visitors = [i[:-1] for i in step1]

In [ ]:
import numpy as np
res_visitors2 = list(np.zeros(12*6+14)) + res_visitors

In [ ]:
print(len(res_visitors2))
print(res_visitors2[-5:])

In [ ]:
display(months_bin.tail(10))

In [ ]:
months_bin.shape

In [ ]:
months_bin['visitors_12_trail'] = res_visitors2
# months_bin['sim_12_trail'] = temp
display(months_bin.tail(10))

In [ ]:
display(months_bin.tail(15))

In [ ]:
months_bin.registered_12_trail[:-11]

In [ ]:
months_bin.sim_12_trail[:-11]

In [ ]:
#months_bin.to_csv('months_bin_backup.csv')

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import os
sns.set()
sns.set_style("white")
#plt.figure(figsize=(9,6))
fig, ax1 = plt.subplots()
ax1.clear()
ax1.plot(months_bin.year_month[:-11], months_bin.registered_12_trail[:-11], color='red', label='Registered Users');
ax1.plot(months_bin.year_month[:-11], months_bin.sim_12_trail[:-11], color='blue', label='Simulation Users');
#plt.plot(months_bin.year_month,res_visitors2)

ax1.set_title('nanoHUB user growth');

ax1.set_xlim([datetime.datetime(2000,1,1), datetime.datetime(2021,3,1)]);
ax1.set_ylim([0, 35000])
ax1.set_yticks(np.arange(0,35000,5000))
ax1.tick_params(axis='y',labelcolor='red')
# ax1.set_ylabel('Unique Users/Month')

ax2 = ax1.twinx()
# ax2.set_ylabel('Unique Visitors/Month')
ax2.plot(months_bin.year_month[:-11],months_bin.visitors_12_trail[11:],color='green',label='Unique Monthly Visitors')
ax2.tick_params(axis='y',labelcolor='green')
ax2.set_yticks(np.arange(0,2000000+250000,250000))
ax2.set_ylim([0,2000000+250000])
# ax2.legend(loc='center left')
# plt.ylabel('Annualized nanoHUB Visitors');
# ax1.legend(loc='upper left');
cwd = os.getcwd()
plt.savefig(cwd+'/appendixF_plots/overall_user_growth_legend.eps', dpi=500, bbox_inches='tight')
# plt.savefig(cwd+'/appendixF_plots/overall_user_growth_no_legend.png', dpi=500, bbox_inches='tight')

In [ ]:
print(months_bin.registered_12_trail[-13:-12])
print(months_bin.sim_12_trail[-13:-12])
print(months_bin.visitors_12_trail[-2:-1])

In [ ]:
months_bin.year_month[:-12]

## Cumulative classroom/research/self-study

In [ ]:
###
# 1. pull all unique classroom users from SF
# 2. pull all research users from DB2
# 3. pull all contacts from SF
# 4. iteratively pull toolstart df again by year starting from 2000

In [ ]:
## pull all classroom users from SF
db_1 = DB2SalesforceAPI(sf_login_params)

In [ ]:
## pull the list of contacts within each cluster
contacts_in_cluster_df = db_1.query_data('Select Contact__c, Tool_Usage_Cluster__c from ContactToolClusterAssociation__c')

In [ ]:
display(contacts_in_cluster_df.head(2))
print(contacts_in_cluster_df.shape)

In [ ]:
## if SF processing errors
if contacts_in_cluster_df.shape[0] <= 100:
    contacts_in_cluster_df = pd.read_csv(cwd+'/backup_contacts_in_clusters.csv')
    contacts_in_cluster_df = contacts_in_cluster_df.drop(columns=['Unnamed: 0'])
display(contacts_in_cluster_df.head(2))
print(contacts_in_cluster_df.shape)

In [ ]:
unique_contacts_in_clusters = set(contacts_in_cluster_df['Contact__c'].to_list())
print(len(unique_contacts_in_clusters))

In [ ]:
contacts_df = db_1.query_data('Select Id, nanoHUB_user_ID__c from Contact')

In [ ]:
display(contacts_df.head(2))
print(contacts_df.shape)

In [ ]:
## determine self-study users, i.e., unclassified users
# if sf id is in contacts_in_cluster_df['Contact__c'], then it is self-study

self_study = []
clustered = []

sf_contact_ids = contacts_df['Id'].to_list()

import numpy as np
# bool_search = np.in1d(np.array(list(unique_contacts_in_clusters)),np.array(sf_contact_ids))
bool_search = np.in1d(np.array(sf_contact_ids),np.array(list(unique_contacts_in_clusters)))

clustered = np.where(bool_search)[0].tolist()
self_study = np.where(~bool_search)[0].tolist()

In [ ]:
clustered_NH_ids = contacts_df['nanoHUB_user_ID__c'][clustered]
self_study_NH_ids = contacts_df['nanoHUB_user_ID__c'][self_study]

In [ ]:
print(clustered_NH_ids.shape)
print(self_study_NH_ids.shape)

In [ ]:
## need to filter self-study-nh-ids with the researchers

In [ ]:
## pull in research users
import sqlalchemy as sql
sql_query = "select uidNumber from jos_citations_authors where uidNumber != 0"
engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %('wang2506_ro', 'fnVnwcCS7iT45EsA'))
researcher_ids = pd.read_sql_query(sql_query, engine)
display(researcher_ids.head(2))

In [ ]:
researcher_ids2 = researcher_ids.drop_duplicates()['uidNumber']
researcher_ids2 = researcher_ids2.to_list()

In [ ]:
researcher_ids.shape

In [ ]:
len(researcher_ids2)

In [ ]:
sf_contact_ids = contacts_df['nanoHUB_user_ID__c'][self_study]

In [ ]:
sf_contact_ids = sf_contact_ids.reset_index()
sf_contact_ids = sf_contact_ids.drop(columns='index')
display(sf_contact_ids)

In [ ]:
bool_search2 = np.in1d(np.array(sf_contact_ids['nanoHUB_user_ID__c'].to_list()),np.array(researcher_ids2))

In [ ]:
researchers = np.where(bool_search2)[0].tolist()
self_study2 = np.where(~bool_search2)[0].tolist()

In [ ]:
print(len(researchers))
print(len(self_study2))

In [ ]:
clustered_NH_ids = pd.DataFrame(contacts_df['nanoHUB_user_ID__c'][clustered]) # already done
self_study_NH_ids = pd.DataFrame(pd.DataFrame(sf_contact_ids).iloc[self_study2,:])
researcher_NH_ids = pd.DataFrame(pd.DataFrame(sf_contact_ids).iloc[researchers,:])

In [ ]:
researcher_NH_ids

In [ ]:
## loading in toolstart

In [ ]:
# matching ids to usernames
sql_query2 = "select id,username from jos_users"
id_username_df = pd.read_sql_query(sql_query2, engine)
display(id_username_df.head(2))

In [ ]:
usernames_db2 = id_username_df['id'].to_list()

In [ ]:
t_cluster = np.in1d(usernames_db2,clustered_NH_ids['nanoHUB_user_ID__c'])
clustered_NH_username = np.where(t_cluster)[0].tolist()
clustered_NH_username2 = pd.DataFrame(id_username_df['username'][clustered_NH_username])
display(clustered_NH_username2)

In [ ]:
t_research = np.in1d(usernames_db2,researcher_NH_ids['nanoHUB_user_ID__c'])
researcher_NH_username = np.where(t_research)[0].tolist()
researcher_NH_username2 = pd.DataFrame(id_username_df['username'][researcher_NH_username])
display(researcher_NH_username2)

In [ ]:
t_self_study = np.in1d(usernames_db2,self_study_NH_ids['nanoHUB_user_ID__c'])
self_NH_username = np.where(t_self_study)[0].tolist()
self_NH_username2 = pd.DataFrame(id_username_df['username'][self_NH_username])
display(self_NH_username2)

In [ ]:
# clustered_NH_username2;researcher_NH_username2;self_NH_username2
clustered_track = []
self_track = []
researcher_track = []

clustered_track_per = []
self_track_per = []
researcher_track_per = []

In [ ]:
engine_metrics = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub_metrics' \
                                               %('wang2506_ro', 'fnVnwcCS7iT45EsA'))

In [ ]:
## computer has memory limits, so split toolstart into branches
start_year = 2000 #2002 #allows for flexibility as well for future updates

while start_year < 2021:#datetime.datetime.now().year:
    start_year += 1
    end_date = r"'"+str(start_year)+r"-01-01'"
#     print(end_date)
    start_date = r"'"+str(start_year-2)+r"-01-01'"
    
    for i in range(1,13):
        if i < 10:
            start_date = r"'"+str(start_year-1)+r"-0"+str(i)+r"-01'"
            end_date = r"'"+str(start_year)+r"-0"+str(i)+r"-01'"
        else:
            start_date = r"'"+str(start_year-1)+r"-"+str(i)+r"-01'"
            end_date = r"'"+str(start_year)+r"-"+str(i)+r"-01'"
        ## unique active users
#         current = time.time()
#         sql_query_clustered = "select count(distinct user) from toolstart where user in " + str(tuple(clustered_NH_username2['username']))\
#             +" and datetime <= "+end_date+" and datetime >= "+start_date
#         clustered_users = pd.read_sql_query(sql_query_clustered, engine_metrics)
        
#         sql_query_self = "select count(distinct user) from toolstart where user in " + str(tuple(self_NH_username2['username']))\
#             +" and datetime <= "+end_date+" and datetime >= "+start_date
#         self_users = pd.read_sql_query(sql_query_self, engine_metrics)
        
#         sql_query_researcher = "select count(distinct user) from toolstart where user in " + str(tuple(researcher_NH_username2['username']))\
#             +" and datetime <= "+end_date+" and datetime >= "+start_date
#         researcher_users = pd.read_sql_query(sql_query_researcher, engine_metrics)
        
        sql_query_clustered = "select user from toolstart where user in " + str(tuple(clustered_NH_username2['username']))\
            +" and datetime <= "+end_date+" and datetime >= "+start_date
        clustered_users = pd.read_sql_query(sql_query_clustered, engine_metrics)#.drop_duplicates()
        
        sql_query_self = "select user from toolstart where user in " + str(tuple(self_NH_username2['username']))\
            +" and user != 'instanton' and user != 'gridstat' and datetime <= "+end_date+" and datetime >= "+start_date
        self_users = pd.read_sql_query(sql_query_self, engine_metrics)#.drop_duplicates()
        
        sql_query_researcher = "select user from toolstart where user in " + str(tuple(researcher_NH_username2['username']))\
            +" and datetime <= "+end_date+" and datetime >= "+start_date
        researcher_users = pd.read_sql_query(sql_query_researcher, engine_metrics)#.drop_duplicates()

#         sql_query_all =  "select user from toolstart where user not in ('gridstat','instanton')" \
#             +" and datetime <= "+end_date+" and datetime >= "+start_date
#         all_users = pd.read_sql_query(sql_query_all, engine_metrics)#.drop_duplicates()
#         print(time.time() - current)
#         clustered_track.append(clustered_users['count(distinct user)'][0])
#         self_track.append(self_users['count(distinct user)'][0])
#         researcher_track.append(researcher_users['count(distinct user)'][0])

        clustered_track.append(clustered_users.drop_duplicates().shape[0])
        self_track.append(self_users.drop_duplicates().shape[0])
        researcher_track.append(researcher_users.drop_duplicates().shape[0])

#         clustered_track_per.append(clustered_users.shape[0]/all_users.shape[0])
#         self_track_per.append(self_users.shape[0]/all_users.shape[0])
#         researcher_track_per.append(researcher_users.shape[0]/all_users.shape[0])        
        
#         print(clustered_users)
#         print(self_users)
#         print(researcher_users)
    print(start_year)

In [ ]:
months_bin.year_month

In [ ]:
user_breakdown_df = pd.DataFrame()
user_breakdown_df['year_month'] = months_bin['year_month'].to_list()[13:]
user_breakdown_df['clustered_track'] = clustered_track
user_breakdown_df['self_track'] = self_track
user_breakdown_df['researcher_track'] = researcher_track
user_breakdown_df.head(2)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
sns.set()
sns.set_style("whitegrid")

plt.figure(figsize=(9,6))
plt.stackplot(user_breakdown_df.year_month, \
              user_breakdown_df[['clustered_track', 'self_track', 'researcher_track']].to_numpy().T, \
             labels=['classroom', 'self-study', 'research'], \
             colors=['green', 'orange', 'red']);

plt.legend(loc='upper left');
plt.ylabel('12-month trailing total')
plt.xlim([datetime.datetime(2001,1,1), datetime.datetime(2021,1,1)])
plt.ylim([0, 22000])

# plt.savefig('./sim_user_by_type_1.svg', bbox_inches='tight')

plt.show()

In [ ]:
user_breakdown_df.tail(15)

In [ ]:
## check researchers
engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %('wang2506_ro', 'fnVnwcCS7iT45EsA'))

sql_query = "select uidNumber,givenName,surname from jos_citations_authors"
jos_citations_authors = pd.read_sql_query(sql_query, engine)


In [ ]:
jos_citations_authors

In [ ]:
sql_query = 'select id, uid, author from jos_citations'
jos_citations = pd.read_sql_query(sql_query, engine)

In [ ]:
jos_citations

In [ ]:
t1_authors = jos_citations['author'].to_list()

In [ ]:
t1_authors2 = [] #[j.split(';') for i,j in enumerate(t1_authors)]

In [ ]:
for i,j in enumerate(t1_authors):
    try:
        holder = j.split(';')
        if len(holder) > 1: 
            for k,l in enumerate(holder):
                if l[0] == ' ':
                    holder[k] = l[1:]
        t1_authors2.append(holder)
    except:
        garb = True

In [ ]:
t1_authors3 = [item for sublist in t1_authors2 for item in sublist]

In [ ]:
t1_authors3 = set(t1_authors3)

In [ ]:
len(t1_authors3)

In [ ]:
## extract all researchers with nh ids
rw_ids = []
need_search = []
for i,j in enumerate(t1_authors3):
    if '{{' in j:
        rw_ids.append(i)
    else:
        need_search.append(i)

In [ ]:
## from the rw_ids, extract the nanohub userids
# can obtain usernames from the DB2 table
rw_ids2 = np.array(list(t1_authors3))[rw_ids]
for i,j in enumerate(rw_ids2):
    begin = j.index('{{')
    rw_ids2[i] = j[begin+2:-2]

In [ ]:
print(len(rw_ids2))

In [ ]:
# need to obtain usernames
sql_query = "select id,name,username from jos_users where id in "+str(tuple(rw_ids2))
rw_researchers = pd.read_sql_query(sql_query,engine)
display(rw_researchers.head(2))

In [ ]:
## need to obtain the username/ids from those people without nh ids explicitly spec'd in jos_citations
ns2 = np.array(list(t1_authors3))[need_search]
ns2 = ns2[1:]

In [ ]:
tuple(ns2)[:5]

In [ ]:
sql_query = "select id,name,username from jos_users where name in "+str(tuple(ns2))
ns2_researchers = pd.read_sql_query(sql_query,engine)
display(ns2_researchers.head(2))

In [ ]:
## combine the usernames into one data structure
researcher_usernames = rw_researchers['username'].to_list() + ns2_researchers['username'].to_list()

In [ ]:
len(researcher_usernames)

In [ ]:
researcher_track = []
engine_metrics = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub_metrics' \
                                               %('wang2506_ro', 'fnVnwcCS7iT45EsA'))

In [ ]:
## computer has memory limits, so split toolstart into branches
start_year = 2000 #2002 #allows for flexibility as well for future updates

while start_year < 2021:#datetime.datetime.now().year:
    start_year += 1
    end_date = r"'"+str(start_year)+r"-01-01'"
#     print(end_date)
    start_date = r"'"+str(start_year-2)+r"-01-01'"
    
    for i in range(1,13):
        if i < 10:
            start_date = r"'"+str(start_year-1)+r"-0"+str(i)+r"-01'"
            end_date = r"'"+str(start_year)+r"-0"+str(i)+r"-01'"
        else:
            start_date = r"'"+str(start_year-1)+r"-"+str(i)+r"-01'"
            end_date = r"'"+str(start_year)+r"-"+str(i)+r"-01'"

        sql_query_researcher = "select user from toolstart where user in " + str(tuple(researcher_usernames))\
            +" and datetime <= "+end_date+" and datetime >= "+start_date
        researcher_users = pd.read_sql_query(sql_query_researcher, engine_metrics)#.drop_duplicates()

        researcher_track.append(researcher_users.drop_duplicates().shape[0])

    print(start_year)

In [ ]:
user_breakdown_df['researcher_track'] = researcher_track

In [ ]:
sns.set()
sns.set_style("whitegrid")

plt.figure(figsize=(9,6))
# plt.figure(figsize=(6,6))
user_breakdown_df2 = user_breakdown_df.iloc[:-11,:]
plt.stackplot(user_breakdown_df2.year_month, \
              user_breakdown_df2[['clustered_track', 'researcher_track','self_track']].to_numpy().T, \
             labels=['Classroom','Research', 'Unclassified'], \
             colors=['green', 'red', 'orange']); #Self-study

plt.legend(loc='upper left')#,fontsize=14);
plt.ylabel('12-month Trailing Total')#,fontsize=15)

plt.xlim([datetime.datetime(2001,1,1), datetime.datetime(2021,2,1)])#,fontsize=15)

# test = plt.xticks()
# # print(test)
# # print([i.get_text() for i in test[1]])
# plt.xticks(ticks=test[0][1:-1],labels=[str(i) for i in range(2002,2021,2)],fontsize=14)
# # plt.xticks(ticks=['2001:1:1','2021:1:1'])#list(range(2000,2021,2)))#, \
# #     labels=list(range(2000,2021,2)),fontsize=12)

# test_y = plt.yticks()
# plt.yticks(ticks=test_y[0][0:-1],labels=[str(i) for i in range(0,21000,5000)],fontsize=14)

# plt.xlim_labels('')
# plt.ylim([0, 24000],fontsize=15)

# plt.savefig(cwd+'/appendixF_plots/sim_user_by_type.png', dpi=500,bbox_inches='tight')
# plt.savefig(cwd+'/appendixF_plots/sim_user_by_type.eps',bbox_inches='tight')
# import os
# cwd = os.getcwd()
# plt.savefig(cwd+'/appendixF_plots/sim_user_by_type_HICSS.png', dpi=1500,bbox_inches='tight')

plt.show()


In [ ]:
user_breakdown_df2['clustered_track'].to_list()[-5:]

In [ ]:
user_breakdown_df2['clustered_track_per'] = 100*np.divide(user_breakdown_df2['clustered_track'],\
            user_breakdown_df2['clustered_track'].to_numpy()+user_breakdown_df2['researcher_track'].to_numpy()+\
                        user_breakdown_df2['self_track'].to_numpy())
user_breakdown_df2['researcher_track_per'] = 100*np.divide(user_breakdown_df2['researcher_track'],\
            user_breakdown_df2['clustered_track'].to_numpy()+user_breakdown_df2['researcher_track'].to_numpy()+\
                        user_breakdown_df2['self_track'].to_numpy())
user_breakdown_df2['self_track_per'] = 100*np.divide(user_breakdown_df2['self_track'],\
            user_breakdown_df2['clustered_track'].to_numpy()+user_breakdown_df2['researcher_track'].to_numpy()+\
                        user_breakdown_df2['self_track'].to_numpy())

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(user_breakdown_df2.year_month, user_breakdown_df2.clustered_track_per, label='classroom', \
             color='green');
plt.plot(user_breakdown_df2.year_month, user_breakdown_df2.researcher_track_per, label='research', \
             color='red');
plt.plot(user_breakdown_df2.year_month, user_breakdown_df2.self_track_per, label='self-study', \
             color='orange');

plt.ylim([0, 100])
plt.legend(loc='upper right');
plt.ylabel('Percentage (%)');
plt.xlim([datetime.datetime(2001,1,1), datetime.datetime(2021,2,1)])

# plt.savefig(cwd+'/appendixF_plots/sim_user_by_type_per.png', dpi=500,bbox_inches='tight')
plt.savefig(cwd+'/appendixF_plots/sim_user_by_type_per.eps',bbox_inches='tight')

plt.show()

## Figure: active simulation users by geolocation

In [ ]:
## need to do iterative importing once again

In [ ]:
engine_metrics = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub_metrics' \
                                               %('wang2506_ro', 'fnVnwcCS7iT45EsA'))

In [ ]:
country_df = pd.DataFrame()
country_df['year_month'] = months_bin['year_month'].to_list()[13:-11]

In [ ]:
country_df

In [ ]:
usa = []
china = []
eu = []
india = []
other = []
eu_list = ['BE','BG','EL','CZ','DK','DE','EE','IE','ES','FR','HR','IT','CY','LV',\
          'LT','LU','HU','MT','NL','AT','PL','PT','RO','SI','SK','FI','SE','IS',\
           'NO','LI','CH','UK','ME','MK','AL','RS','TR','BA','XK','AM','AZ','BY',\
           'MD','GE','UA','RU']

In [ ]:
## computer has memory limits, so split toolstart into branches
start_year = 2000 #2002 #allows for flexibility as well for future updates

while start_year < 2021:#datetime.datetime.now().year:
    start_year += 1
#     end_date = r"'"+str(start_year)+r"-01-01'"
# #     print(end_date)
#     start_date = r"'"+str(start_year-1)+r"-01-01'"
    
    for i in range(1,13):
        if i < 10:
            start_date = r"'"+str(start_year-1)+r"-0"+str(i)+r"-01'"
            end_date = r"'"+str(start_year)+r"-0"+str(i)+r"-01'"
        else:
            start_date = r"'"+str(start_year-1)+r"-"+str(i)+r"-01'"
            end_date = r"'"+str(start_year)+r"-"+str(i)+r"-01'"

        sql_query_country = "select distinct user, countryip from toolstart where user != 'instanton' "\
        + "and user != 'gridstat' and datetime <= "+end_date+" and datetime >= "+start_date
        country_users = pd.read_sql_query(sql_query_country, engine_metrics)#.drop_duplicates()
        
        # run geography filter
        country_ipds = country_users['countryip'].to_list()
        
        bool_US = np.in1d(np.array(country_ipds),np.array(['US']))
        bool_EU = np.in1d(np.array(country_ipds),np.array(eu_list))
        bool_CN = np.in1d(np.array(country_ipds),np.array(['CN']))
        bool_IN = np.in1d(np.array(country_ipds),np.array(['IN']))
#         bool_OT = bool_US | bool_EU | bool_CN | bool_IN
        
        usa.append(len(np.where(bool_US)[0].tolist()))
        eu.append(len(np.where(bool_EU)[0].tolist()))
        china.append(len(np.where(bool_CN)[0].tolist()))
        india.append(len(np.where(bool_IN)[0].tolist()))
        other.append(len(country_ipds)-usa[-1]-eu[-1]-china[-1]-india[-1]) # len(np.where(bool_OT)[0].tolist()))
        
#         country_users.append(country_users.drop_duplicates().shape[0])

    print(start_year)

In [ ]:
country_df.shape

In [ ]:
usa[-8:]

In [ ]:
print(len(usa))
usa2 = usa[37:-8]
print(len(usa2))

In [ ]:
country_df['USA'] = usa[37:-8]
country_df['China'] = china[37:-8]
country_df['Europe'] = eu[37:-8]
country_df['India'] = india[37:-8]
country_df['Other'] = other[37:-8]

In [ ]:
country_df

In [ ]:
country_df['Total'] = np.array(usa[37:-8])+np.array(china[37:-8])+np.array(eu[37:-8])+np.array(india[37:-8])+np.array(other[37:-8])

In [ ]:
plt.figure(figsize=(9,6))
plt.stackplot(country_df['year_month'], \
              country_df[['USA', 'China', 'India', 'Europe', 'Other']].to_numpy().T, \
             labels=country_df.columns.to_list()[1:-1], \
             colors=['blue', 'red', 'orange', 'green', 'black']);

plt.legend(loc='upper left');
plt.ylabel('12-month trailing total')
plt.xlim([datetime.datetime(2002,1,1), datetime.datetime(2021,2,1)]);
plt.ylim([0, 25000])

# plt.savefig(cwd+'/appendixF_plots/sim_user_by_country.png', dpi=500,bbox_inches='tight')
plt.savefig(cwd+'/appendixF_plots/sim_user_by_country.eps',bbox_inches='tight')

plt.show()

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(country_df['year_month'], \
         country_df['USA'].to_numpy()/country_df['Total'].to_numpy() *100\
         , label='US', color='blue');
plt.plot(country_df['year_month'], \
         country_df['China'].to_numpy()/country_df['Total'].to_numpy() *100\
         , label='China', color='red');
plt.plot(country_df['year_month'], \
         country_df['India'].to_numpy()/country_df['Total'].to_numpy() *100\
         , label='India', color='orange');
plt.plot(country_df['year_month'], \
         country_df['Europe'].to_numpy()/country_df['Total'].to_numpy() *100\
         , label='Europe', color='green');
plt.plot(country_df['year_month'], \
         country_df['Other'].to_numpy()/country_df['Total'].to_numpy() *100\
         , label='Others', color='black');

plt.xlim([datetime.datetime(2001,1,1), datetime.datetime(2021,2,1)])
plt.ylim([0, 100])
plt.legend(loc='upper right');
plt.ylabel('Percentage (%)')

# plt.savefig(cwd+'/appendixF_plots/sim_user_by_country_perc.png', dpi=500,bbox_inches='tight')
plt.savefig(cwd+'/appendixF_plots/sim_user_by_country_perc.eps',bbox_inches='tight')

plt.show()

## Classroom activity plots

In [ ]:
## students and classes per semester

In [ ]:
## pull all classroom users from SF
db_1 = DB2SalesforceAPI(sf_login_params)

In [ ]:
## pull the list of contacts within each cluster
contacts_in_cluster_df = db_1.query_data('Select Id, Contact__c, Tool_Usage_Cluster__c from ContactToolClusterAssociation__c')

In [ ]:
contacts_in_cluster_df

In [ ]:
contacts_in_cluster_df = pd.read_csv(cwd+'/backup_contacts_in_clusters.csv')
contacts_in_cluster_df = contacts_in_cluster_df.drop(columns=['Unnamed: 0'])
contacts_in_cluster_df

In [ ]:
clusters_df = db_1.query_data('Select id, ID__c, Semester__c, Ending_Date__c, Starting_Date__c from tool_usage_cluster__c')
clusters_df

In [ ]:
clusters_df = pd.read_csv(cwd+'/backup_tool_cluster_df.csv')
clusters_df = clusters_df.drop(columns=['Unnamed: 0'])
clusters_df

In [ ]:
## filter by starting_date__c
cluster_dts = pd.to_datetime(clusters_df['Starting_Date__c'])

## don't use current year's data
cluster_dts2 = clusters_df[cluster_dts <= '2021-01-01']
cluster_dts2 = cluster_dts2.reset_index()['index'].to_list()
display(cluster_dts2[:5])

In [ ]:
clusters_df2 = clusters_df.iloc[cluster_dts2,:]

In [ ]:
clusters_df2 = clusters_df2.reset_index().drop(columns='index')

In [ ]:
clusters_df2

In [ ]:
## calculate classes per semester
clusters_df2['Starting_Date__c'] = pd.to_datetime(clusters_df2['Starting_Date__c'])

starting_year = 2000

semester_list = ['Spring','Fall','Summer']
# group Spring and Summer togethre

months_sems = []

while starting_year < 2022:
    for i in semester_list:
        if i == 'Fall':
            months_sems.append(datetime.datetime.strptime(str(starting_year)+'-07-01', '%Y-%m-%d') )
        elif i == 'Spring':
            months_sems.append(datetime.datetime.strptime(str(starting_year)+'-01-01', '%Y-%m-%d') )
    starting_year += 1

In [ ]:
cluster_ids_per_sem = {}
for i,j in enumerate(months_sems):
    if i == 0:
        cluster_ids_per_sem[i] = clusters_df2['Id'].to_numpy()[clusters_df2['Starting_Date__c'] < months_sems[i+1]]
    elif i == len(months_sems)-1:
        cluster_ids_per_sem[i] = clusters_df2['Id'].to_numpy()[(clusters_df2['Starting_Date__c'] > j)]
    else:
        cluster_ids_per_sem[i] = clusters_df2['Id'].to_numpy()[(clusters_df2['Starting_Date__c'] > j) \
                        & (clusters_df2['Starting_Date__c'] < months_sems[i+1])]

In [ ]:
## produce classes sizes per semester
cluster_ids_per_sem_nums = [len(i) for i in cluster_ids_per_sem.values()]

In [ ]:
# # find overlaps
# ttt = np.in1d(np.array(clusters_df['Id']),np.array(contacts_in_cluster_df['Tool_Usage_Cluster__c']))
# len(ttt)

In [ ]:
## find the student populations per semester
students = []
students_count = []
for i in cluster_ids_per_sem.values():
    bool_students = np.in1d(np.array(contacts_in_cluster_df['Tool_Usage_Cluster__c'].to_list()),np.array(i))
    
    students.append(contacts_in_cluster_df['Contact__c'].to_numpy()[np.where(bool_students)[0]]) #dataframe indexes
    students_count.append(len(students[-1]))

In [ ]:
months_sems2 = [str(i.year)+'-0'+str(i.month) for i in months_sems]

In [ ]:
## plot two bar plots
plt.figure(figsize=(9,6))

g = sns.barplot(months_sems2, cluster_ids_per_sem_nums, color='blue');
g.set_xticklabels(months_sems2, rotation=90)

plt.xlabel('Semester')
plt.ylabel('Classes')

# plt.savefig(cwd+'/appendixF_plots/class_by_semester_count.png', dpi=500,bbox_inches='tight')
plt.savefig(cwd+'/appendixF_plots/class_by_semester_count.eps',bbox_inches='tight')

plt.figure(figsize=(9,6))
g = sns.barplot(months_sems2, students_count, color='green');
g.set_xticklabels(months_sems2, rotation=90)

plt.xlabel('Semester')
plt.ylabel('Students')

# plt.savefig(cwd+'/appendixF_plots/student_by_semester_count.png', dpi=500, bbox_inches='tight')
plt.savefig(cwd+'/appendixF_plots/student_by_semester_count.eps', bbox_inches='tight')

In [ ]:
## determine cumulative uniques
#students
cumulative_sem_students = []
unique_students = []
cumulative_classes = []

for i,j in enumerate(students):
    if i == 0:
        unique_students.append(len(set(j)))
        cumulative_sem_students.append(students_count[i])
        cumulative_classes.append(cluster_ids_per_sem_nums[i])
    elif i == len(students)-1:
        unique_students.append(len(set(np.concatenate(students[:i+1]))))
        cumulative_sem_students.append(students_count[i] + cumulative_sem_students[-1])
        cumulative_classes.append(cluster_ids_per_sem_nums[i] + cumulative_classes[-1])
    else:
        unique_students.append(len(set(np.concatenate(students[:i+1]))))
        cumulative_sem_students.append(students_count[i]+cumulative_sem_students[-1])  
        cumulative_classes.append(cluster_ids_per_sem_nums[i] + cumulative_classes[-1])

In [ ]:
starting_year = 2001

semester_list = ['Spring','Fall','Summer']
# group Spring and Summer togethre

months_sems3 = [datetime.datetime(2000, 7, 1, 0, 0)]

while starting_year < 2022:
    for i in semester_list:
        if i == 'Fall':
            months_sems3.append(datetime.datetime.strptime(str(starting_year)+'-07-01', '%Y-%m-%d') )
        elif i == 'Spring':
            months_sems3.append(datetime.datetime.strptime(str(starting_year)+'-01-01', '%Y-%m-%d') )
    starting_year += 1

In [ ]:
len(months_sems3[:-1])

In [ ]:
np.concatenate(students)

In [ ]:
duplicate_students = list(np.concatenate(students))

In [ ]:
from itertools import groupby
intermed_student = [len(list(group)) for key, group in groupby(duplicate_students)]
np.where(np.array(intermed_student) > 1)

In [ ]:
counter = 1
for key,group in groupby(duplicate_students):
    counter += 1
print(counter)

In [ ]:
print(cumulative_sem_students[-1],unique_students[-1],cumulative_classes[-1])
print(cluster_ids_per_sem_nums[-1],len(set(np.concatenate(students[-1:]))))

In [ ]:
## percent diff
(89730-60357)/89730

In [ ]:
# plot cumulative class and student count
fig, ax1 = plt.subplots(figsize=(9,6))

color = 'tab:red'
ax1.set_ylabel('Persons', color=color)
#ax1.plot(class_students_count.cumsum(), color=color)
ax1.plot(months_sems3[:-1], \
         cumulative_sem_students, color='Green')
ax1.plot(months_sems3[:-1], unique_students, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Classes', color=color)  # we already handled the x-label with ax1
ax2.plot(months_sems3[:-1],cumulative_classes, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.xlim([datetime.datetime(2000,7,1), datetime.datetime(2021,1,1)])

ax1.set_ylim([0, 95000])
ax1.grid(False)
ax2.set_ylim([0, 8000])
ax2.grid(False)

# plt.savefig(cwd+'/appendixF_plots/student_class_count.png', dpi=500, bbox_inches='tight')
plt.savefig(cwd+'/appendixF_plots/student_class_count.eps', bbox_inches='tight')

plt.show()

## User Lifetime Comparison

In [ ]:
#pull jos_users with the previously found self-study, classroom, and research users
#then just compare the register vs last visit date

In [ ]:
sql_query = "select registerDate, lastvisitDate from jos_users where username in " + str(tuple(clustered_NH_username2['username']))
engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %('wang2506_ro', 'fnVnwcCS7iT45EsA'))

In [ ]:
clustered_dates = pd.read_sql_query(sql_query,engine)
clustered_dates

In [ ]:
clustered_dates[clustered_dates['lastvisitDate'].isna()].shape[0]

In [ ]:
## determine time diffs
clustered_dates2 = clustered_dates[~clustered_dates['lastvisitDate'].isna()].reset_index()
clustered_dates2['registerDate'] = pd.to_datetime(clustered_dates2['registerDate'])
clustered_dates2['lastvisitDate'] = pd.to_datetime(clustered_dates2['lastvisitDate'])
clustered_dates2['duration'] = clustered_dates2['lastvisitDate'].to_numpy() - clustered_dates2['registerDate'].to_numpy()
display(clustered_dates2.head(2))

In [ ]:
clustered_dates2['duration'] = [np.round(np.log10(i.days),2) for i in clustered_dates2['duration'].to_list()]
display(clustered_dates2.head(2))
display(clustered_dates2.tail(2))

In [ ]:
# 50 bins - log days
max_log_interval = 4 #base 10
bins = 50
# 1 day, 2 day, 3 day, 4, 5, 6, 1 week, 2 week, 1 month, 2 month, 1 year, 
bin_vals = [np.round(4/50*(i+1),2) for i in range(bins)] #these are the ending bin vals
bin_vals2 = [np.round(4/50*(i+1),2) for i in range(bins)] #these are the ending bin vals

In [ ]:
clustered_user_vals = []
for i,j in enumerate(bin_vals):
    if i == 0:
#         clustered_user_vals.append(clustered_dates[clustered_dates['lastvisitDate'].isna()].shape[0]\
#                                   +len(np.where(clustered_dates2['duration'] == 1)[0]))
        h_thresh = j
        l_thresh = 0
        clustered_user_vals.append(len(np.where((clustered_dates2['duration'] < h_thresh))[0]))
    else:
        h_thresh = j
        l_thresh = bin_vals[i-1]
        clustered_user_vals.append(len(np.where((clustered_dates2['duration'] > l_thresh) \
                                    & (clustered_dates2['duration'] < h_thresh))[0]))        

        

In [ ]:
clustered_user_vals[0]

In [ ]:
clustered_user_vals2 = []
for i in clustered_user_vals:
    if i != 0:
        clustered_user_vals2.append(np.log10(i))
    else:
        clustered_user_vals2.append(0)

In [ ]:
plt.figure()
plt.hist(bin_vals2,bins=bin_vals2,weights=clustered_user_vals,align='left',color='green')
plt.xlabel('User Lifetime (log days)')
plt.xticks(np.arange(0,4.5,0.5))
plt.ylim([0,2500])
plt.ylabel('# of Users (log)')


plt.savefig('classroom_users_hist.eps',bbox_inches='tight')

In [ ]:
sql_query = "select registerDate, lastvisitDate from jos_users where username in " + str(tuple(self_NH_username2['username']))
engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %('wang2506_ro', 'fnVnwcCS7iT45EsA'))

In [ ]:
self_study_dates = pd.read_sql_query(sql_query,engine)
self_study_dates

In [ ]:
## determine time diffs
self_study_dates2 = self_study_dates[~self_study_dates['lastvisitDate'].isna()].reset_index()
self_study_dates2['registerDate'] = pd.to_datetime(self_study_dates2['registerDate'])
self_study_dates2['lastvisitDate'] = pd.to_datetime(self_study_dates2['lastvisitDate'])
self_study_dates2['duration'] = self_study_dates2['lastvisitDate'].to_numpy() - self_study_dates2['registerDate'].to_numpy()
display(self_study_dates2.head(2))

In [ ]:
self_study_dates2['duration'] = [np.round(np.log10(i.days),2) for i in self_study_dates2['duration'].to_list()]
display(self_study_dates2.head(2))
display(self_study_dates2.tail(2))

In [ ]:
self_user_vals = []
for i,j in enumerate(bin_vals):
    if i == 0:
#         clustered_user_vals.append(clustered_dates[clustered_dates['lastvisitDate'].isna()].shape[0]\
#                                   +len(np.where(clustered_dates2['duration'] == 1)[0]))
        h_thresh = j
        l_thresh = 0
        self_user_vals.append(len(np.where((self_study_dates2['duration'] < h_thresh))[0]))
    else:
        h_thresh = j
        l_thresh = bin_vals[i-1]
        self_user_vals.append(len(np.where((self_study_dates2['duration'] > l_thresh) \
                                    & (self_study_dates2['duration'] < h_thresh))[0]))        

        

In [ ]:
self_user_vals[:5]

In [ ]:
plt.figure()
plt.hist(bin_vals2,bins=bin_vals2,weights=self_user_vals,align='left',color='orange')
plt.xlabel('User Lifetime (log days)')
plt.xticks(np.arange(0,4.5,0.5))
plt.ylim([0,2500])
plt.ylabel('# of Users')


plt.savefig('self_users_hist.eps',bbox_inches='tight')

In [ ]:
sql_query = "select registerDate, lastvisitDate from jos_users where username in " + str(tuple(researcher_usernames))
engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %('wang2506_ro', 'fnVnwcCS7iT45EsA'))

In [ ]:
researcher_dates = pd.read_sql_query(sql_query,engine)
researcher_dates

In [ ]:
## determine time diffs
researcher_dates2 = researcher_dates[~researcher_dates['lastvisitDate'].isna()].reset_index()
researcher_dates2['registerDate'] = pd.to_datetime(researcher_dates2['registerDate'])
researcher_dates2['lastvisitDate'] = pd.to_datetime(researcher_dates2['lastvisitDate'])
researcher_dates2['duration'] = researcher_dates2['lastvisitDate'].to_numpy() - researcher_dates2['registerDate'].to_numpy()
display(researcher_dates2.head(2))

In [ ]:
researcher_dates2['duration'] = [np.round(np.log10(i.days),2) for i in researcher_dates2['duration'].to_list()]
display(researcher_dates2.head(2))
display(researcher_dates2.tail(2))

In [ ]:
researcher_vals = []
for i,j in enumerate(bin_vals):
    if i == 0:
#         clustered_user_vals.append(clustered_dates[clustered_dates['lastvisitDate'].isna()].shape[0]\
#                                   +len(np.where(clustered_dates2['duration'] == 1)[0]))
        h_thresh = j
        l_thresh = 0
        researcher_vals.append(len(np.where((researcher_dates2['duration'] < h_thresh))[0]))
    else:
        h_thresh = j
        l_thresh = bin_vals[i-1]
        researcher_vals.append(len(np.where((researcher_dates2['duration'] > l_thresh) \
                                    & (researcher_dates2['duration'] < h_thresh))[0]))        

        

In [ ]:
researcher_vals[:2]

In [ ]:
plt.figure()
plt.hist(bin_vals2,bins=bin_vals2,weights=researcher_vals,align='left',color='red')
plt.xlabel('User Lifetime (log days)')
plt.xticks(np.arange(0,4.5,0.5))
plt.ylim([0,200])
plt.ylabel('# of Users')


plt.savefig('research_users_hist.eps',bbox_inches='tight')

In [ ]:
clustered_NH_username2['username'].to_csv("clustered_users.csv")

In [ ]:
self_NH_username2['username'].to_csv('self_study_users.csv')

In [ ]:
tt_temp = pd.DataFrame()
tt_temp['usernames'] = researcher_usernames
tt_temp.to_csv('researchers.csv')